In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, TensorDataset
import gymnasium as gym

from astropy.io import fits
from datetime import datetime, timezone
import pandas as pd
import json
import fitsio
import time
import pickle
import re

%reload_ext autoreload
%autoreload 2

In [5]:
import survey_ops
from survey_ops.utils import units, geometry, interpolate
from survey_ops.coreRL.offline_dataset import OfflineDECamDataset
from survey_ops.coreRL.agents import Agent
from survey_ops.algorithms import DDQN, BehaviorCloning
from survey_ops.utils.sys_utils import seed_everything
from survey_ops.coreRL.data_processing import load_raw_data_to_dataframe


In [6]:
from survey_ops.utils import ephemerides
from tqdm import tqdm
from pathlib import Path

In [7]:
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import KMeans
from collections import Counter
from scipy.stats import entropy

In [8]:
SEED = 10
seed_everything(SEED)
torch.set_default_dtype(torch.float32)
device = torch.device(
    "cuda" if torch.cuda.is_available() else
    "cpu"   
)

# Load config and lookup files

In [40]:
results_outdir = '../experiment_results/bc-radec-grid-test/'

In [41]:
with open('../configs/global_config.json', 'r') as f:
    gcfg = json.load(f)

In [42]:
with open(results_outdir + 'config.json', 'r') as f:
    cfg = json.load(f)

In [43]:
nside = cfg['data']['nside']

In [47]:
cfg['data']['specific_years'] = [2015]

In [63]:
with open(gcfg['paths']['LOOKUP_DIR'] + '/' + gcfg['files']['FIELD2RADEC'], 'r') as f:
    field2radec = json.load(f)
with open(gcfg['paths']['LOOKUP_DIR'] + '/' + gcfg['files']['FIELD2NAME'], 'r') as f:
    field2name = json.load(f)
with open(gcfg['paths']['LOOKUP_DIR'] + '/' + gcfg['files']['FIELD2NVISITS'], 'r') as f:
    field2nvisits = json.load(f)
with open(f'../data/lookups/nside{nside}_bin2azel.json', 'r') as f:
    bin2azel = json.load(f)
with open(f'../data/lookups/nside{nside}_bin2radec.json', 'r') as f:
    bin2radec = json.load(f)


In [64]:
# fits_path = Path(cfg.get('paths.DFITS')).resolve().parents[1] / 'data' / cfg.get('paths.DFITS')
# json_path = Path(cfg.get('paths.DJSON')).resolve().parents[1] / 'data' / cfg.get('paths.DJSON')

df = load_raw_data_to_dataframe(fits_path=None, json_path='../data/fits/decam-exposures-20251211.json')

d = OfflineDECamDataset(
    df=df,
    cfg=cfg,
    gcfg=gcfg,
)

Calculating zenith states: 100%|███████████| 124/124 [00:00<00:00, 50206.94it/s]
Calculating sun and moon ra/dec and az/el: 100%|█| 14100/14100 [00:11<00:00, 119
Calculating bin features for all healpix bins and timestamps: 100%|█| 14100/1410
Normalizing bin features: 100%|████████| 12288/12288 [00:00<00:00, 22686.24it/s]


In [50]:
df['field_id'] = df['object'].map({v: k for k, v in field2name.items()})

In [69]:
from collections import defaultdict

[autoreload of survey_ops.coreRL.data_processing failed: Traceback (most recent call last):
  File "/home/hurra/.conda/envs/cosmoML/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 325, in check
    superreload(m, reload, self.old_objects)
  File "/home/hurra/.conda/envs/cosmoML/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 580, in superreload
    module = reload(module)
             ^^^^^^^^^^^^^^
  File "/home/hurra/.conda/envs/cosmoML/lib/python3.11/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 621, in _exec
  File "<frozen importlib._bootstrap_external>", line 936, in exec_module
  File "<frozen importlib._bootstrap_external>", line 1074, in get_code
  File "<frozen importlib._bootstrap_external>", line 1004, in source_to_code
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "/home/hurra/Projects/survey-ops/survey_ops/coreRL/d

In [70]:
field_visit_counter = defaultdict(int)

In [73]:
field_visit_counter[10] += 1

In [77]:
field_visit_counter[[100, 20, 30]]

TypeError: unhashable type: 'list'

In [78]:
np.array([1, 2, 3]).min()

np.int64(1)

In [83]:
field2radec['14313']

[5.899411859222181, -0.00019891517484979374]

In [81]:
field2name['14313']

'DES survey hex -219-4 tiling 10'

In [79]:
field2name[14313]

KeyError: 14313

In [16]:
df

,expnum,ra,dec,exptime,filter,propid,program,object,teff,fwhm,datetime,az,zd,ha,airmass,qc_fwhm,qc_cloud,qc_sky,qc_teff,timestamp
0,229250,301.199042,-50.938332,90,g,2012B-0001,survey,DES survey hex -579-501 tiling 3,0.41,1.15,2013-08-31 23:55:07,139.5500,32.21,-33.208458,1.18,1.15,0.42,1.26,0.41,1377993307
1,229251,300.851117,-52.967276,90,g,2012B-0001,survey,DES survey hex -582-521 tiling 3,0.60,1.18,2013-08-31 23:57:08,143.3200,32.76,-32.380917,1.19,1.18,0.15,1.23,0.60,1377993428
2,229252,300.460079,-54.996221,90,g,2012B-0001,survey,DES survey hex -586-542 tiling 3,0.62,1.17,2013-08-31 23:59:08,146.9500,33.42,-31.494500,1.20,1.17,-0.07,1.22,0.62,1377993548
3,229253,300.016742,-57.024887,90,g,2012B-0001,survey,DES survey hex -590-562 tiling 3,0.52,1.29,2013-09-01 00:01:11,150.4500,34.21,-30.536042,1.21,1.29,-0.21,1.20,0.52,1377993671
4,229254,302.722079,-58.040193,90,g,2012B-0001,survey,DES survey hex -563-572 tiling 3,0.44,1.40,2013-09-01 00:03:13,150.5900,35.76,-32.738375,1.23,1.40,0.04,1.20,0.44,1377993793
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88613,810366,58.930033,-42.303110,90,r,2012B-0001,des_celeb,NGC 1487,0.55,1.35,2019-01-10 04:27:33,238.7702,38.66,46.306458,1.28,1.35,0.09,-0.15,0.55,1547094453
88614,810367,58.929917,-42.303138,90,i,2012B-0001,des_celeb,NGC 1487,0.51,1.16,2019-01-10 04:29:31,238.7969,39.02,46.803625,1.29,1.16,0.08,0.19,0.51,1547094571
88615,810370,53.414579,-36.057110,45,i,2012B-0001,des_celeb,NGC 1365,0.30,1.41,2019-01-10 04:35:39,247.6877,44.83,53.840917,1.41,1.41,0.08,0.34,0.30,1547094939
88616,810371,53.389708,-36.223610,45,g,2012B-0001,des_celeb,NGC 1365,0.46,1.59,2019-01-10 04:36:53,247.3923,45.09,54.204458,1.42,1.59,0.12,-0.18,0.46,1547095013


In [17]:
df['field_id']

KeyError: 'field_id'